# COVID-19 Predictings using LSTM RNN

This notebook is used to compare outcomes of deep learning methods to mathematical SIR model variants.

Deep learning methods require a lot of data as input for training. However, training one single model for all countries across the world raises a lot of problems. Not all countries show the same trend in infections, testing and registration is different for multiple countries, data shows a lot of fluctuations etc. 

A solution to this problem is to train a model countries' data with similar trends. Training a country is therefore done by using similar countries as training input. 

## Preparation

In [1]:
import os
import warnings 
warnings.filterwarnings('ignore')
os.chdir('..')

In [2]:
import time
import torch
import statistics

import numpy   as np 
import pandas  as pd

import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.metrics       import mean_squared_log_error
from sklearn.preprocessing import StandardScaler

from tqdm             import tqdm
from IPython.display  import display 

from torch import nn
from torch import optim
from torch.optim import lr_scheduler

In [3]:
from core.data      import compare_countries as cc
from core.data      import utils             as dataUtils

from core.nn        import WeightInitializer
from core.nn.loss   import rmsle_error, l1_norm_error
from core.nn.loss   import GradientSmoothLoss

from core.networks  import BasicRecurrentPredictor

%load_ext autoreload
%autoreload 2

In [4]:
# Reproduceble results
torch.manual_seed(123);
torch.cuda.manual_seed(123)
np.random.seed(123)
torch.backends.cudnn.deterministic = True

### Global variables

In [5]:

DEVICE       = 'cpu'
TRAIN_UP_TO  = pd.to_datetime('2020-10-01')
RNNNOCELLS = 2
# LEARNING_RATE = 0.05
CELLTYPE = "LSTMCell"
ITERATE = 5
WINDOWSIZE = 5
FUTURESTEPS = 123


## Data

Data is collected from the [GitHub](https://github.com/CSSEGISandData/COVID-19) repository  created by Johns Hopkins CSSE.

In [6]:
COUNTRY = "Netherlands"
allData = pd.read_csv('./assets/covid_spread.csv', parse_dates=['Date'])
allData.head()


,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,NaN,Afghanistan,2020-01-22,0.0,0.0
1,NaN,Afghanistan,2020-01-23,0.0,0.0
2,NaN,Afghanistan,2020-01-24,0.0,0.0
3,NaN,Afghanistan,2020-01-25,0.0,0.0
4,NaN,Afghanistan,2020-01-26,0.0,0.0


In [7]:
# Add data to Province_State
if COUNTRY is "global":
    print("Converting")
    allData["Province_State"] = "-"
    allData["Country_Region"] = "global"
    allData = dataUtils.preprocess_data(allData)
    
    # Replace Confirmed cases with new confirmed cases
    allData = allData.sort_values("Date")
    allData = allData.rename(columns={"Confirmed": "ConfirmedCases", "Fatal": "Fatalities"})
    
    allData = allData.reset_index()[columns]
    allData["New Confirmed"] = allData.groupby("Country_Region")["ConfirmedCases"].diff()
    columns = allData.columns
    allData = allData.groupby('Country_Region', as_index=False).apply(lambda group: group.iloc[1:])
else:
    # Add data to Province_State
    allData = dataUtils.preprocess_data(allData)
    
    allData = allData.sort_values("Date")
    allData["New Confirmed"] = allData.groupby("Country_Region")["ConfirmedCases"].diff()
    columns = allData.columns
    allData = allData.groupby('Country_Region', as_index=False).apply(lambda group: group.iloc[1:])
allData.head()

Province_State Country_Region       Date  ConfirmedCases  Fatalities  \
0 1    Afghanistan    Afghanistan 2020-01-23             0.0         0.0   
  2    Afghanistan    Afghanistan 2020-01-24             0.0         0.0   
  3    Afghanistan    Afghanistan 2020-01-25             0.0         0.0   
  4    Afghanistan    Afghanistan 2020-01-26             0.0         0.0   
  5    Afghanistan    Afghanistan 2020-01-27             0.0         0.0   

     New Confirmed  
0 1            0.0  
  2            0.0  
  3            0.0  
  4            0.0  
  5            0.0

### Similar country training data

Select the data using a threshold. This threshold only selects countries higher than this value. Then, the smallest error is computed by the sliding-window method for every step. Data from the selected country is slided across, to compute the error at each step.  

The result is a dataframe containing the following:
- **Province_State** : the name of the state/province
- **deathError**     : the minimum error that was found between source and that specific country for fatalities
- **deathIdx**       : the index where the above error was found (in the thresholded array)
- **confirmedError** : the minimum error that was found between source and that specific country for confirmed cases
- **confirmedIdx**   : the index where the above error was found (in the thresholded array)

**MAPE** (mean absolute percenta error) it used as the comparing measure. This represents the error in percentage relative to the source country, hence being easier to interpret and understand.  


In [8]:
errorData  = cc.get_nearest_sequence(allData, COUNTRY, 
                                     alignThreshConf = 80,
                                     alignThreshDead = 20,  
                                     errorFunc       = rmsle_error
                                    )

display(errorData.sort_values(by='confirmedError').head())
display(errorData.sort_values(by='deathError').head())

,Province_State,deathError,confirmedError,deathIdx,confirmedIdx
5,Japan,0.779142,0.436392,2,14
3,Iran,0.442889,0.467222,0,0
4,Italy,0.690287,0.530056,0,0
2,France,0.659473,0.553949,0,0
1,Hubei,0.228588,0.571953,1,0


,Province_State,deathError,confirmedError,deathIdx,confirmedIdx
1,Hubei,0.228588,0.571953,1,0
3,Iran,0.442889,0.467222,0,0
7,Spain,0.646557,0.640611,0,0
2,France,0.659473,0.553949,0,0
4,Italy,0.690287,0.530056,0,0


In [9]:
errorData

,Province_State,deathError,confirmedError,deathIdx,confirmedIdx
0,Henan,2.407042,1.939461,15,2
1,Hubei,0.228588,0.571953,1,0
2,France,0.659473,0.553949,0,0
3,Iran,0.442889,0.467222,0,0
4,Italy,0.690287,0.530056,0,0
5,Japan,0.779142,0.436392,2,14
6,"Korea, South",1.215604,0.818127,13,13
7,Spain,0.646557,0.640611,0,0
8,US,1.305151,1.556039,0,0
9,United Kingdom,0.766020,0.579083,0,0


### Creating traning data

Training data is selected including only the countries that have an average error smaller that a threshold. This is done for both confirmed cases and fatalities. These models are trained seperately. 

In [15]:
confData = dataUtils.get_target_data(allData, errorData, 
                                     errorThresh = 0.5, 
                                     country     = COUNTRY, 
                                     target      = 'confirmed')
deadData = dataUtils.get_target_data(allData, errorData, 
                                     errorThresh = .5, 
                                     country     = COUNTRY, 
                                     target      = 'fatalities')

confData = confData[["Date", "Province_State", "ConfirmedCases"]]
deadData = deadData[["Date", "Province_State", "Fatalities"]]

confData.head()

,Date,Province_State,ConfirmedCases
"(87, 56211)",2020-01-23,Japan,2.0
"(87, 56212)",2020-01-24,Japan,2.0
"(87, 56213)",2020-01-25,Japan,2.0
"(87, 56214)",2020-01-26,Japan,4.0
"(87, 56215)",2020-01-27,Japan,4.0


[StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) is used to scale the dates because a small standard deviation is expected for features. This scaler will be further used for scaling both the training data and the model predictions.

In [16]:
confScaler = dataUtils.get_scaler(confData, 'confirmed')
deadScaler = dataUtils.get_scaler(deadData, 'fatalities')

## Predicting

### Model

The reccurent predictor is composed from 2 parts:
- reccurent cells  
- multi layer perceptron that is applied after each encoded timpestamp 
Each part will be individually configured.

Two individual models are trained for the two features, Confirmed Cases and Fatalities. 

An overview of the model is shown in this figure:

![RNN_Flow](../assets/images/rnn_model.png)

#### Parameters

These define overall shapes of the data and the size of the embeddings:
- **chNo**   : the number of input and output features, they must have the same length, in out case this       
- **future** : the number of timestapms to predict into the feature 

#### RNN parameters

- **rnnCell** : RNN cell types that are supported in Pytorch (LSTM/GRU/RNN), the most powerfull one is LSTM, though, having a small amount of data(like in our case) GRU may be better choice thus is generalize better. RNNCell is also an option yet, but it yielded the worst results
- **rnnNoCells** : represent the number of reccurent cells in the model a higher number leads to a very unstable model (especially for this task) and the exploding gradients problem occurs very often during training 
- **hidChNo** : number of RNN cell hidden dimension

**Note**: yes, this part could have been implemented using the higher level and more optimized [API](https://pytorch.org/docs/stable/nn.html#torch.nn.LSTM) from PyTorch, but I think this is more expressive and easier to understand and it is not such does not require a lot more code

#### MLP parameters

- **mlpLayerCfg** : a list with the numbers of neurons in each hidden layer. The layers might be wideer but is to be avoided having a very deep network (numerical instability during training and no real benefit from it)
- **mlpActiv** : activation layer after each dense/linear layer, I have used [PReLU](https://pytorch.org/docs/stable/nn.html#prelu) which is a sort of learnable loss 
- **dropRate** : dropout rate applied after each dense/linear layer, for our case 0 is used
- **normType** : normalization layer, not used used
- **mlpActivLast** : last layer activation function 

## Training

Because the both Confirmed and Fatalities are going to follow the same lead and only a few parameters will vary, I will only explain once everything and add a few notes if that will be necessarly. 

In [17]:
w = WeightInitializer()

### Confirmed

For this section we have more data, since the cases have started earlier and the fatalities came later on. 

#### Initialize Confirmed Cases Model

In [18]:
# build the model
confModel = BasicRecurrentPredictor(
            # parameters
            chNo        = 1,          # number of input features
            future      = 0,
            returnFullSeq = True,     # return both the encoded sequence 
                                      # and the future prediction
    
            # RNN
            rnnCell     = CELLTYPE, # RNN cell type (LSTM/GRU/RNN)
            rnnNoCells  = RNNNOCELLS,          # no of RNN cells
            hidChNo     = 12,         # number of RNN cell hidden dimension
            
            # MLP
            mlpLayerCfg   = [4],      # layer hidden dims
            mlpActiv      = 'PReLU',  # inner activation of the mlp
            dropRate      = None,     # dropout rate for each layer of mlp
            normType      = None,     # normalization type
            mlpActivLast  = None      # note that every timestamp 
                                      # in the sequence will be activated too
            
            ).build()

Moving on, the model will have the weights initialized as as follows:
- RNN : default (provided by PyTorch)
- MLP : normal distribution

In [19]:
w.init_weights(confModel, 'normal_', {})

#### Timing constants and data splitting

![data_model](../assets/images/data_diagram.png)

Constants:
- **winSize**     - the  size of observation period and prediction period 
- **obsSize**     - number of timestemps used for observation
- **futureSteps** - the number of predicted days, not including the observation period
- **supPredSteps** - are the number predicted days that are going to be optimized in a supervised manner
- **uPredSteps** - are the number predicted days that are going to be optimized in an unsupervised manner
- **allPredSteps** - the total length of the prediction output including the observation period and the future steps

In [20]:
winSize       = WINDOWSIZE
obsSize       = 4
futureSteps   = FUTURESTEPS
supPredSteps  = winSize - obsSize
uPredSteps    = futureSteps - supPredSteps
allPredSteps  = futureSteps + obsSize

#### Train Data

Split the dataframe from the previously selected data into batches and normalize it. The data is only selected until a specific date, and the rest is left for validation.  

In [21]:
confTrainData = dataUtils.get_train_data(confData, 'confirmed', 
                                  step       = 1,
                                  winSize    = winSize, 
                                  trainLimit = TRAIN_UP_TO, 
                                  scaler     = confScaler,
                                  shuffle    = True)
confTrainData.shape

torch.Size([682, 5, 1])

#### Loss

$$ \mathcal{L}_{total} = \mathcal{L}_{sup} + \mathcal{L}_{unsup} $$

##### Supervised loss

[SmoothL1Loss](https://pytorch.org/docs/stable/nn.html#torch.nn.SmoothL1Loss) (Huber Loss) is used for training. It is less sensitive to outliers than MSELoss and prevents exploding gradients.

In [22]:
confLoss  = nn.SmoothL1Loss()

##### Unsupervised loss

It may not be very straight forward from data that the output should somehow resemble a logistic curve, so do this we are trying to optimize this in an unsupervised manner. A smoothing loss, is implied which will limit the growth of a function over a large periods. 

To better scale this loss and to make it more stable, insead of scaling the final result I chose clipping the values before averaging them. 

Please note that the the length of the predicton and the clipping values are considered hyperparameters, and they should be tuned for each country.

In [23]:
gradsTrain  = confTrainData[:, 1:] - confTrainData[:, :-1] 
confGradMax = gradsTrain.max()

confGLoss   = GradientSmoothLoss(confGradMax, uPredSteps)

#### Optimizer

As optimization algorithm the LBFGS one is chosed, it is a very memory expensive algorithm, but since we have such little data, it works just fine. For more details about how it works I leave the following link on [Quora](https://www.quora.com/Is-the-L-bfgs-always-better-than-stochastic-gradient-descent?share=1).

In [24]:
# confOptim = optim.Adam(confModel.parameters(), 
#                         lr             = 0.05, 
#                       )

confOptim = optim.LBFGS(confModel.parameters(), 
                        lr             = 0.05, 
                        max_iter       = 75, 
                        tolerance_grad = 1e-7, 
                        history_size   = 75
                      )

#### Device 

In [25]:
confModel.to(DEVICE);
confTrainData = confTrainData.to(DEVICE);

#### Closure

The closure is the function that is passed to the optimizers. Here is the "bussines logic" of the training process. For the first OBS_SIZE steps, the models prediction is based on the input data, from there, it uses it's own prediction. The whole sequence is returned and evaluted for optimization. 

For better stability, after the loss is computed, the gradients are clipped such that their L2 norm is 1. For more details check this [article](https://machinelearningmastery.com/exploding-gradients-in-neural-networks/) by Jason Brownlee.

![image.png](../assets/images/model_data.png)

In [26]:
def conf_closure():
    confOptim.zero_grad()
    confModel.returnFullSeq = True
    
    # slice data
    obsData = confTrainData[:,:obsSize]
    
    # make prediction
    out  = confModel(obsData, future = futureSteps)
    out  = out.reshape(-1, allPredSteps, 1)
    
    # compute gradients
    loss = confLoss(out[:, :winSize], confTrainData)
    
    # unsupervised loss
    smoothLoss = confGLoss(out[:,winSize:], 0.25)
    loss += smoothLoss 
    
    # make prediciton follow an ascending trend
    # by forcing the gradients to be positie (still testing)
    grads = out[:, 1:] - out[:, :-1]
    grads[grads > 0] = 0
    grads = grads.mean().abs()
    loss += grads
    loss.backward()
    
    # clip gradients / numerical stability
    nn.utils.clip_grad_norm_(confModel.parameters(), 1.0)
      
    return loss

#### Train loop

Training loop is the section where the parameters are optimized.

In [27]:
pBar = tqdm(range(ITERATE))
for i in pBar:
    loss = confOptim.step(conf_closure)
    
    # update tqdm to show loss and lr
    pBar.set_postfix({'Loss ' : loss.item(), 
                      'Lr'    : confOptim.param_groups[0]['lr']})
    
    if torch.isnan(loss):
        raise ValueError('Loss is NaN')

  0%|          | 0/5 [00:29<?, ?it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-7a935a26bcf2>", line 3, in <module>
    loss = confOptim.step(conf_closure)
  File "/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/torch/autograd/grad_mode.py", line 15, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/torch/optim/lbfgs.py", line 437, in step
    loss = float(closure())
  File "/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/torch/autograd/grad_mode.py", line 15, in decorate_context
    return func(*args, **kwargs)
  File "<ipython-input-26-ce16e6e9037f>", line 9, in conf_closure
    out  = confModel(obsData, future = futureSteps)
  File "/usr/local/Caskroom/miniconda/base/lib/python3.7/site-packages/torch/nn/modules/module.p

KeyboardInterrupt: 

### Evaluate

The evaluation process is done for the source country only. For this, the last OBS_SIZE timestemps from the train data are fed into the model, and the output is compared with data from validation period. 

In [ ]:
confValData, confValLabel = dataUtils.get_val_data(confData, 'confirmed', 
                                                   COUNTRY, 
                                                   TRAIN_UP_TO, 
                                                   obsSize, 
                                                   confScaler)
confValData = confValData.to(DEVICE)



For evaluating the prediction, MAPE is used. The error is only computed on the predicted future and not on the predicted observation period. 

In [ ]:

confModel.eval()
# get figure
fig, ax = plt.subplots(1, 1, figsize = (9, 4))
ax.tick_params(axis='x', rotation=45)
fig.suptitle(COUNTRY + ' confirmed cases prediction')

# make prediction
confModel.returnFullSeq = False
pred   = confModel(confValData, future = futureSteps).cpu().detach().numpy()
pred   = confScaler.inverse_transform(pred[0])

error  = rmsle_error(pred[:confValLabel.shape[0]], confValLabel.numpy())
print("RMSLE : %2.5f"% error.item(), ' (not normalized)')             

# prediction
predDate = pd.date_range(start = TRAIN_UP_TO, periods=pred.shape[0])              
sns.lineplot(y = pred, x = predDate, ax = ax, linewidth=4.5)


# plot train data
showTrainData = confData[confData['Province_State'] == COUNTRY]
showTrainData = showTrainData[showTrainData['Date'] < TRAIN_UP_TO]
sns.lineplot(y = 'ConfirmedCases', x = 'Date', data = showTrainData, ax = ax, linewidth=4.5)

# plot val data
showValData = confData[confData['Province_State'] == COUNTRY]
showValData = showValData[showValData['Date'] >= TRAIN_UP_TO]
sns.lineplot(y = 'ConfirmedCases', x ='Date', data = showValData, ax = ax, linewidth=4.5);

# SIR = pd.read_csv('assets/SIR_global.csv', parse_dates=['Date'])
# sns.lineplot(x="Date", y="Confirmed", data=SIR, ax = ax, linewidth=4.5)

# SIRD = pd.read_csv('assets/SIRD_global.csv', parse_dates=['Date'])
# sns.lineplot(x="Date", y="Confirmed", data=SIRD, ax = ax, linewidth=4.5)

# SIRF = pd.read_csv('assets/SIRF_global.csv', parse_dates=['Date'])
# sns.lineplot(x="Date", y="Confirmed", data=SIRF, ax = ax, linewidth=4.5)


# ax.legend(['LSTM', 'GRU', 'True', 'Train', "SIR", "SIRD", "SIRF"])
ax.legend(['LSTM', 'True', 'Train'])
ax.axvline(x=TRAIN_UP_TO, ymin = 0.0, ymax = 1.0, linestyle='--', lw = 1, color = '#808080')
ax.grid(True)


In [ ]:


# df1 = pd.DataFrame(pred, index=predDate, columns=["RNN"])
# df8 = pd.DataFrame(pred_LSTM, index=predDate, columns=["LSTM"])
# df2 = pd.DataFrame(pred_GRU, index=predDate, columns=["GRU"])
# df3 = pd.DataFrame(SIRF["Fatal"].values, index=SIRF["Date"].values, columns=["SIRF"])
# df4 = pd.DataFrame(SIR["Confirmed"].values, index=SIR["Date"].values, columns=["SIR"])
# df5 = pd.DataFrame(SIRD["Confirmed"].values, index=SIRD["Date"].values, columns=["SIRD"])
# df6 = pd.DataFrame(showTrainData["Fatalities"].values, index=showTrainData["Date"].values, columns=["train"])
# df7 = pd.DataFrame(showValData["Fatalities"].values, index=showValData["Date"].values, columns=["val"])

# merged = df1.join(df2)
# merged = merged.join(df3)
# merged = merged.join(df4)
# merged = merged.join(df5)
# merged = merged.join(df8)
# merged = merged.join(df6, how="outer")
# merged = merged.join(df7, how="outer")
# merged

# # merged2 = df6.merge(df7)
# # merged2
# merged.to_pickle("global_comparison_fatalities.p")

### Fatalities

Because all the explication were done in the previous section, here cleaner view of the code is provided.

#### Model

Initialize the model

In [ ]:
# build the model
deadModel = BasicRecurrentPredictor(
            # basic parameters
            chNo          = 1,              # number of input features
            future        = 0,
            returnFullSeq = True,
    
            # RNN
            rnnCell       = 'LSTMCell',     # RNN cell type (LSTM/GRU/RNN)
            rnnNoCells    = 1,              # no of RNN cells
            hidChNo       = 16,             # number of RNN cell hidden dimension
    
            # MLP
            mlpLayerCfg   = [4],            # layer hidden dims
            mlpActiv      = 'PReLU',        # inner activation of the mlp
            dropRate      = None,           # dropout rate for each layer of mlp
            normType      = None,           # normalization type
            mlpActivLast  = None,           # note that every timestamp in the sequence 
                                            # will be activated too
            ).build()

Initialize models weights.

In [ ]:
w.init_weights(deadModel, 'normal_', {})

#### Timming constants

#### Train data

In [ ]:
deadTrainData = dataUtils.get_train_data(deadData, 'fatalities', 
                                      step       = 1,
                                      winSize    = winSize, 
                                      trainLimit = TRAIN_UP_TO, 
                                      scaler     = deadScaler,
                                      shuffle    = True)
deadTrainData.shape

#### Loss

In [ ]:
deadLoss  = nn.SmoothL1Loss()

In [ ]:
gradsTrain = deadTrainData[:, 1:] - deadTrainData[:, :-1] 
deadGradMax = gradsTrain.max()

deadGLoss = GradientSmoothLoss(deadGradMax, uPredSteps)

#### Optimizer

In [ ]:
deadOptim = optim.LBFGS(deadModel.parameters(), 
                        lr             = 0.05, 
                        max_iter       = 75, 
                        tolerance_grad = 1e-7, 
                        history_size   = 75)

#### Device

In [ ]:
deadModel.to(DEVICE);
deadTrainData = deadTrainData.to(DEVICE);

#### Closure

In [ ]:
def dead_closure():
    deadOptim.zero_grad()
    
    # slice data
    obsData  = deadTrainData[:,:obsSize]

    out  = deadModel(obsData, future = futureSteps)
    out  = out.reshape(-1, allPredSteps, 1)
 
    # compute and backprop loss
    loss = deadLoss(out[:, :winSize], deadTrainData)   
    
    # smooth
    smoothLoss = deadGLoss(out[:,winSize:], 0.25)
    loss += smoothLoss
    
    # make prediciton follow an ascending trend
    # (still testing)
    grads = out[:, 1:] - out[:, :-1]
    grads[grads > 0] = 0
    grads = grads.mean().abs()
    loss += grads
    loss.backward()
    
    # clip gradient for numerical stability
    nn.utils.clip_grad_norm_(deadModel.parameters(), 1.0)
    
    return loss

#### Train loop

In [ ]:
pBar = tqdm(range(10))
for i in pBar:
    loss = deadOptim.step(dead_closure)
    pBar.set_postfix({'Loss ' : loss.item()})
    
    if torch.isnan(loss):
        raise ValueError('Loss is NaN')

### Evaluate

In [ ]:
deadValData, deadValLabel = dataUtils.get_val_data(deadData, 'fatalities', 
                                                   COUNTRY, 
                                                   TRAIN_UP_TO, 
                                                   obsSize, 
                                                   deadScaler)
deadValData = deadValData.to(DEVICE)

In [ ]:
## deadModel.eval()

# get figure
fig, ax = plt.subplots(1, 1, figsize = (9, 4))
ax.tick_params(axis='x', rotation=45)
fig.suptitle(COUNTRY + ' fatalities prediction')

# make prediction
deadModel.returnFullSeq = False
pred   = deadModel(deadValData, future = futureSteps).cpu().detach().numpy()
pred   = deadScaler.inverse_transform(pred[0])
error  = rmsle_error(pred[:deadValLabel.shape[0]], deadValLabel.numpy())
print('RMSLE: ', error)             

# plot prediction
predDate = pd.date_range(start = TRAIN_UP_TO, periods=pred.shape[0])              
sns.lineplot(y = pred, x = predDate, ax = ax, linewidth=4.5 )

# plot train data
showTrainData = deadData[deadData['Province_State'] == COUNTRY]
showTrainData = showTrainData[showTrainData['Date'] < TRAIN_UP_TO]
sns.lineplot(y = 'Fatalities', x = 'Date', data = showTrainData, ax = ax, linewidth=4.5)

# plot val data
showValData = deadData[deadData['Province_State'] == COUNTRY]
showValData = showValData[showValData['Date'] >= TRAIN_UP_TO]
sns.lineplot(y = 'Fatalities', x ='Date', data = showValData, ax = ax, linewidth=4.5);

ax.legend(['Pred', 'Train', 'Validation'])
ax.axvline(x=TRAIN_UP_TO, ymin = 0.0, ymax = 1.0, linestyle='--', lw = 1, color = '#808080')
ax.grid(True)

In [ ]:
# deadData.to_csv("fatalities_train.csv")
# showValData.to_csv("fatalities_val.csv")
showValData
# # get figure
# fig, ax = plt.subplots(1, 1, figsize = (9, 4))
# ax.tick_params(axis='x', rotation=45)
# fig.suptitle(COUNTRY + ' fatalities prediction')

# # make prediction
# deadModel.returnFullSeq = False
# pred   = deadModel(deadValData, future = futureSteps).cpu().detach().numpy()
# pred   = deadScaler.inverse_transform(pred[0])
# error  = rmsle_error(pred[:deadValLabel.shape[0]], deadValLabel.numpy())
# print('RMSLE: ', error)             

# # plot prediction
# predDate = pd.date_range(start = TRAIN_UP_TO, periods=pred.shape[0])              
# sns.lineplot(y = pred, x = predDate, ax = ax, linewidth=4.5 )

# # plot train data
# showTrainData = deadData[deadData['Province_State'] == COUNTRY]
# showTrainData = showTrainData[showTrainData['Date'] < TRAIN_UP_TO]
# sns.lineplot(y = 'Fatalities', x = 'Date', data = showTrainData, ax = ax, linewidth=4.5)

# # plot val data
# showValData = deadData[deadData['Province_State'] == COUNTRY]
# showValData = showValData[showValData['Date'] >= TRAIN_UP_TO]
# sns.lineplot(y = 'Fatalities', x ='Date', data = showValData, ax = ax, linewidth=4.5);

# SIR = pd.read_csv('assets/SIR_global.csv', parse_dates=['Date'])
# SIR["Recovered"] = SIR["Recovered"] / 25
# sns.lineplot(x="Date", y="Recovered", data=SIR, ax = ax, linewidth=4.5)

# SIRD = pd.read_csv('assets/SIRD_global.csv', parse_dates=['Date'])
# SIRD["Recovered"] = SIRD["Recovered"] / 25
# sns.lineplot(x="Date", y="Recovered", data=SIRD, ax = ax, linewidth=4.5)

# SIRF = pd.read_csv('assets/SIRF_global.csv', parse_dates=['Date'])
# SIRF["Recovered"] = SIRF["Recovered"] / 25
# sns.lineplot(x="Date", y="Recovered", data=SIRF, ax = ax, linewidth=4.5)

# ax.legend(['LSTM', 'True', 'Train', "SIR", "SIRD", "SIRF"])
# ax.axvline(x=TRAIN_UP_TO, ymin = 0.0, ymax = 1.0, linestyle='--', lw = 1, color = '#808080')
# ax.grid(True)